# Model Training Anatomy

https://huggingface.co/docs/transformers/model_memory_anatomy

# Import

The nvidia-ml-py3 library allows us to monitor the memory usage of the models from within Python. You might be familiar with the nvidia-smi command in the terminal - this library allows to access the same information in Python directly.

In [1]:
import numpy as np
from pynvml import *
from datasets import Dataset

In [2]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [3]:
print_gpu_utilization()

GPU memory occupied: 624 MB.


When a model is loaded to the GPU the kernels are also loaded, which can take up 1-2GB of memory. To see how much it is we load a tiny tensor into the GPU which triggers the kernels to be loaded as well.

In [4]:
import torch


torch.ones((1, 1)).to("cuda")

print_gpu_utilization()

GPU memory occupied: 746 MB.


# Load Model

First, we load the bert-large-uncased model. We load the model weights directly to the GPU so that we can check how much space just the weights use.

In [5]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")

print_gpu_utilization()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory occupied: 2037 MB.


# Anatomy of Model's operations

Transformers architecture includes 3 main groups of operations grouped below by compute-intensity.

1. Tensor Contractions
2. Statistical Normalizations
3. Element-wise Operators

Linear layers and components of Multi-Head Attention all do batched matrix-matrix multiplications. These operations are the most compute-intensive part of training a transformer.

Softmax and layer normalization are less compute-intensive than tensor contractions, and involve one or more reduction operations, the result of which is then applied via a map.

These are the remaining operators: biases, dropout, activations, and residual connections. These are the least compute-intensive operations.

[Data Movement Is All You Need: A Case Study on Optimizing Transformers 2020](https://arxiv.org/abs/2007.00072)

# Anatomy of Model's Memory

We’ve seen that training the model uses much more memory than just putting the model on the GPU. This is because there are many components during training that use GPU memory. The components on GPU memory are the following:

1. model weights
2. optimizer states
3. gradients
4. forward activations saved for gradient computation
5. temporary buffers
6. functionality-specific memory

A typical model trained in mixed precision with AdamW requires **18 bytes per model parameter** plus activation memory.

For inference there are no optimizer states and gradients, so we can subtract those. And thus we end up with 6 bytes per model parameter for mixed precision inference, plus activation memory.

In [ ]:
seq_len, dataset_size = 512, 512

dummy_data = {

    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),

    "labels": np.random.randint(0, 1, (dataset_size)),

}

ds = Dataset.from_dict(dummy_data)

ds.set_format("pt")

# Methods and tools for efficient training on a single GPU

https://huggingface.co/docs/transformers/perf_train_gpu_one

When training large models, there are two aspects that should be considered at the same time:

* Data throughput (débit)/training time
* Model performance

Maximizing the throughput (samples/second) leads to lower training cost. This is generally achieved by utilizing the GPU as much as possible and thus filling GPU memory to its limit. If the desired batch size exceeds the limits of the GPU memory, the memory optimization techniques, such as gradient accumulation, can help.

However, if the preferred batch size fits into memory, there’s no reason to apply memory-optimizing techniques because they can slow down the training. Just because one can use a large batch size, does not necessarily mean they should. As part of hyperparameter tuning, you should determine which batch size yields the best results and then optimize resources accordingly.

#  Batch size choice

To achieve optimal performance, start by identifying the appropriate batch size. It is recommended to use batch sizes and input/output neuron counts that are of size 2^N. Often it’s a multiple of 8, but it can be higher depending on the hardware being used and the model’s dtype.